In [4]:
import requests
import json
import time
import copy

with open("key.txt", 'r') as f:
    key = f.read()
print(key)

3CA293064B9ECCDC101642C75CCD74DE


In [5]:
#This will only get us recent 500 matches
def retrieve_recent_match_data(key, start_at=None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key + "&min_players=10")
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" 
                         + str(start_at) + "&key=" + key + "&min_players=10")
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)

In [6]:
private_id = 4294967295 #players that we can not get info from
unique_id = set([])
least_recent = None
leave_loop = False
while leave_loop is False:
    matches = retrieve_recent_match_data(key, start_at = least_recent)
    if matches['result']['results_remaining'] == 0:
        leave_loop = True
    for m in matches['result']['matches']:
        least_recent = m['match_id']
        for p in m['players']:
            if 'account_id' in p and p['account_id'] != private_id:
                unique_id.add(p['account_id'])
    time.sleep(0.9)
    print(len(unique_id))

347
665
665


In [7]:
def find_player_recent_match(key, player_id, start_at = None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key + 
                         "&min_players=10&account_id="+str(player_id))
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" 
                         + str(start_at) + "&key=" + key + "&min_players=10&account_id="+str(player_id))
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)

In [8]:
temp_set = copy.deepcopy(unique_id)
for account_id in temp_set:
    leave_loop = False
    least_recent = None
    while leave_loop is False:
        matches = find_player_recent_match(key, account_id, start_at = least_recent)
        if matches['result']['results_remaining'] == 0:
            leave_loop = True
        for m in matches['result']['matches']:
            least_recent = m['match_id']
            for p in m['players']:
                if 'account_id' in p and p['account_id'] != private_id:
                    unique_id.add(p['account_id'])
        time.sleep(0.9)
        print(len(unique_id))
    if len(unique_id) > 100000:
        break

770
855
1460
2073
2729
3418
4061
4092
4517
5039
5551
6099
6651
6671
6709
6745
6784
6828
6867
6868
7351
7840
7909
8097
8312
8558
8812
9030
9040
9278
9531
9750
10026
10306
10316
10652
10964
11274
11604
11935
11955
12192
12400
12620
12861
13127
13145
13337
13534
13742
13928
14094
14103
14320
14603
14922
15211
15471
15483
16052
16590
17222
17825
18437
18459
18720
18876
19025
19178
19384
19394
19875
20336
20859
21443
22048
22071
22578
23050
23521
24053
24535
24553
24863
25165
25422
25692
25958
25967
26455
26829
27113
27308
27517
27530
27970
28382
28783
29197
29562
29578
29917
30324
30695
31010
31349
31362
31805
32306
32800
33286
33759
33776
34230
34728
35286
35719
36145
36150
36635
37098
37612
38107
38642
38659
38794
38963
39113
39309
39502
39509
40083
40689
41249
41772
42234
42258
42656
43035
43435
43792
44230
44247
44850
45359
45913
46217
46447
46719
46998
47312
47593
47601
48002
48471
48873
49310
49780
49803
50209
50648
51085
51563
52087
52108
52440
52734
53030
53265
53422
53593
53804
54

In [9]:
with open("player_id.txt", 'w') as f:
    for p in unique_id:
        f.write(str(p))
        f.write("\n")